In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame,Series
import seaborn as sns
%matplotlib inline

In [8]:
full = pd.read_excel("Round2_Dataset_G1.xlsx")
data = full.loc[full["Response"]=="Yes",]

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8294 entries, 0 to 8293
Data columns (total 24 columns):
Customer                         8294 non-null object
State                            8294 non-null object
Customer Lifetime Value          8294 non-null float64
Response                         8294 non-null object
Coverage                         8294 non-null object
Education                        8294 non-null object
Effective To Date                8294 non-null object
EmploymentStatus                 8293 non-null object
Gender                           8293 non-null object
Income                           8293 non-null float64
Location Code                    8293 non-null object
Marital Status                   8293 non-null object
Monthly Premium Auto             8293 non-null float64
Months Since Last Claim          8293 non-null float64
Months Since Policy Inception    8293 non-null float64
Number of Open Complaints        8293 non-null float64
Number of Policies     

In [6]:
data.loc[pd.isnull(data["Income"])==True,]

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
8293,NL87371,Oregon,2435.245577,No,Basic,Bachelor,1/,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data["Response"].value_counts()

No     7089
Yes    1205
Name: Response, dtype: int64

In [8]:
y = data["Response"]

In [9]:
y.ravel()

array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype=object)

In [10]:
x = data.drop("Response",axis=1)

In [11]:
np.array(x).ravel().shape

(190762,)

In [12]:
np.dstack([,x])

SyntaxError: invalid syntax (<ipython-input-12-76d0daa9e3c7>, line 1)

In [16]:
np.random.random(10000).reshape(100,100)

array([[ 0.36435436,  0.90647479,  0.56310564, ...,  0.57326923,
         0.9179585 ,  0.71760949],
       [ 0.34893764,  0.14253362,  0.42497075, ...,  0.06889127,
         0.79176862,  0.52031229],
       [ 0.74077955,  0.27734623,  0.88431259, ...,  0.93841641,
         0.01993397,  0.05613643],
       ..., 
       [ 0.39763633,  0.77446511,  0.40344524, ...,  0.41035251,
         0.65012454,  0.72064005],
       [ 0.50943429,  0.23293743,  0.77481259, ...,  0.87356649,
         0.34018932,  0.75270364],
       [ 0.80314183,  0.99436858,  0.72432792, ...,  0.71224027,
         0.26011864,  0.54227837]])

In [3]:
from sklearn.neighbors import NearestNeighbors
def nearestneighbor(sample):
    nbrs = NearestNeighbors(algorithm='ball_tree',n_neighbors=6,leaf_size=100)
    nbrs.fit(sample)
    index = nbrs.kneighbors(sample,return_distance=False)
    return index
    

In [72]:
nearestneighbor(data[data.describe(0).columns].dropna())

array([[   0, 2832, 4538, 4144, 6556, 4741],
       [   1, 1284, 4202,  986, 4133,  435],
       [   2,  648, 1009,  383, 4313, 4015],
       ..., 
       [8290, 3645, 6881, 3682, 8009, 2236],
       [8291, 7578, 3129,  267, 2096, 4877],
       [8292, 4662, 4345,  697, 2007, 3730]])

In [4]:
cont = data.describe(0).columns
cat = data.select_dtypes([object]).columns
median = np.median(np.std(data[cont]))

In [26]:
def nearest(x,sample):
    euc = {}
    factor = []
    for i,j in sample.iterrows():
        
        #number of times to add median
        mul = 0
        
        for p,q in j[cat].iteritems():
           
            #if categorical feature are different, add the mean square
            if q != x[p]:
                mul+=1

        #the euclidean distance between the two rows
        euc[i] = np.sqrt(sum((j[cont]-x[cont])**2 + (mul*(median**2))))
    
    #return indices with min euclidean distance
    return [(x,euc[x]) for x in sorted(euc, key=euc.get)[:6]]
    #return factor

In [162]:
def nearall(data):
    final = []
    sample = data.dropna()
    for i,j in data.dropna().iterrows():
        final.append((i,nearest(j,sample)))
    return final

In [27]:
nearest(data.iloc[0],data.dropna())

[(5, 0.0),
 (4375, 128.09784842500596),
 (3569, 128.09784842500596),
 (3873, 128.09784842500596),
 (7787, 688.04446212942298),
 (7307, 688.04446212942298)]

In [7]:
data.iloc[5]

Customer                                     MB98372
State                                         Oregon
Customer Lifetime Value                      2771.05
Response                                         Yes
Coverage                                       Basic
Education                                    College
Effective To Date                2011-11-01 00:00:00
EmploymentStatus                            Employed
Gender                                             F
Income                                         50071
Location Code                                  Rural
Marital Status                                Single
Monthly Premium Auto                              71
Months Since Last Claim                            3
Months Since Policy Inception                     32
Number of Open Complaints                          0
Number of Policies                                 1
Policy Type                           Corporate Auto
Policy                                  Corpor

In [9]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(algorithm="ball_tree",n_neighbors=6,leaf_size=50,metric="euclidean")
nbrs.fit(data[cont].dropna())

NearestNeighbors(algorithm='ball_tree', leaf_size=50, metric='euclidean',
         metric_params=None, n_jobs=1, n_neighbors=6, p=2, radius=1.0)

In [65]:
final = data
for i in cat.drop(["Effective To Date","Customer"])[1:]:
    final = pd.concat([final,median*pd.get_dummies(data[i],drop_first=True)],axis=1)
final.reset_index(inplace=True,drop=True)
final.drop(cat,axis=1,inplace=True)

In [67]:
def newnearest(x):
    nbrs = NearestNeighbors(n_neighbors=6,algorithm="ball_tree",leaf_size=100,metric="euclidean")
    nbrs.fit(x)
    return nbrs.kneighbors(x,return_distance=False)

In [68]:
index = newnearest(final)

In [69]:
index = index.T[1:].T

In [75]:
index

array([[ 648,  553,  515, 1062, 1128],
       [ 741,   67,  815,  379,  271],
       [ 817,    4,  107,  320,  691],
       ..., 
       [ 543,  258,  305,   58,  670],
       [1163, 1060,  324,  173,  710],
       [ 523, 1057,  493,  937,  868]])

In [143]:
cat2 = cat.drop(["Effective To Date","Customer"])

Index(['Customer', 'State', 'Response', 'Coverage', 'Education',
       'Effective To Date', 'EmploymentStatus', 'Gender', 'Location Code',
       'Marital Status', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Vehicle Class', 'Vehicle Size'],
      dtype='object')

In [ ]:
def synthetic(n,index,final):
    if n<100:
        t* = n/100
        n=100
    n=n//100
    
    select = [(random.sample(list(value),2)) for value in index]
    for i,x in zip(final.iterrows(),select):
        for p,q in j[cont].iteritems():
            dic={}
            for y in x:
                gap = random.randrange(0,10)/10
                dic[p[0]] = p[1]+ gap*(p[1]-final.iloc[y])
                
                
            
        
        
    

In [ ]:
def synthetic(n,index,sample,t):
    if n<100:
        t* = n/100
        n=100
    n=n//100    
    
    for i,x in zip(sample.iterrows(),select):
        dic = {}
        it=0
    
    
        for y in x:
            gap = random.randrange(0,10)/10
            dic = dict(gap*(i[1][cont]-sample[cont].iloc[y]) +i[1][cont])
    
    
        ind = data.iloc[index[it]][cat2].mode().T.index
        val = data.iloc[index[it]][cat2].mode().T.values
        for l,m in zip(ind,val):
            dic[l]=m[0]
            
        dic["Customer"] = "SYN" + str(it)
        
        it += 1
        
        

In [32]:
k=5
t=final.shape[1]
n = 200
send_data= data.reset_index(drop=True)


1

In [55]:
import random
select = [(random.sample(list(value),2)) for value in index]

TypeError: random_sample() takes at most 1 positional argument (2 given)

In [151]:
for i,x in zip(final.iterrows(),select):
    dic = {}
    it=0
    
    
    for y in x:
        gap = random.randrange(0,10)/10
        dic = dict(gap*(i[1][cont]-final[cont].iloc[y]) +i[1][cont])
    
    
    ind = data.iloc[index[it]][cat2].mode().T.index
    val = data.iloc[index[it]][cat2].mode().T.values
    for i,j in zip(ind,val):
        dic[i]=j[0]
    it += 1
    
    break
                

In [101]:
for i,x in zip(final.iterrows(),select):
    gap = random.randrange(0,10)/10
    print(dict(gap*(i[1][cont]-final[cont].iloc[x[0]]) +i[1][cont]))
    break

{'Total Claim Amount': -33.25603000000001, 'Customer Lifetime Value': 7934.2449896000007, 'Number of Open Complaints': 0.0, 'Number of Policies': 0.39999999999999991, 'Monthly Premium Auto': 37.0, 'Income': 63252.400000000001, 'Months Since Policy Inception': 133.19999999999999, 'Months Since Last Claim': 6.7999999999999998}


In [140]:
Series(dic)
#add policy, customer no and effective date

Coverage                                 Basic
Customer Lifetime Value                8135.53
Education                             Bachelor
EmploymentStatus                      Employed
Gender                                       F
Income                                 63033.4
Location Code                            Rural
Marital Status                         Married
Monthly Premium Auto                        57
Months Since Last Claim                   11.3
Months Since Policy Inception            108.7
Number of Open Complaints                    0
Number of Policies                         1.4
Policy Type                      Personal Auto
Renew Offer Type                        Offer2
Response                                   Yes
Sales Channel                              Web
State                                  Arizona
Total Claim Amount                     87.1434
Vehicle Class                     Two-Door Car
Vehicle Size                           Medsize
dtype: object

In [147]:
data.iloc[[0,1]].append(Series(dic),ignore_index=True)

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,OC83172,Oregon,8256.297800,Yes,Basic,Bachelor,1/25/11,Employed,F,62902.0,...,94.0,0.0,2.0,Personal Auto,Personal L3,Offer2,Web,159.383042,Two-Door Car,Medsize
1,XZ87318,Oregon,5380.898636,Yes,Basic,College,2/24/11,Employed,F,55350.0,...,13.0,0.0,9.0,Corporate Auto,Corporate L3,Offer1,Agent,321.600000,Four-Door Car,Medsize
2,NaN,Arizona,8135.527996,Yes,Basic,Bachelor,NaN,Employed,F,63033.4,...,108.7,0.0,1.4,Personal Auto,NaN,Offer2,Web,87.143390,Two-Door Car,Medsize


In [148]:
data.head(1000)

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
5,OC83172,Oregon,8256.297800,Yes,Basic,Bachelor,1/25/11,Employed,F,62902.0,...,94.0,0.0,2.0,Personal Auto,Personal L3,Offer2,Web,159.383042,Two-Door Car,Medsize
6,XZ87318,Oregon,5380.898636,Yes,Basic,College,2/24/11,Employed,F,55350.0,...,13.0,0.0,9.0,Corporate Auto,Corporate L3,Offer1,Agent,321.600000,Four-Door Car,Medsize
8,DY87989,Oregon,24127.504020,Yes,Basic,Bachelor,1/26/11,Medical Leave,M,14072.0,...,3.0,0.0,2.0,Corporate Auto,Corporate L3,Offer1,Agent,511.200000,Four-Door Car,Medsize
13,SJ95423,Arizona,8819.018934,Yes,Basic,High School or Below,2011-10-01 00:00:00,Employed,M,99845.0,...,25.0,1.0,8.0,Corporate Auto,Corporate L3,Offer2,Branch,528.000000,SUV,Medsize
22,YH23384,Arizona,24127.504020,Yes,Basic,Bachelor,1/26/11,Medical Leave,M,14072.0,...,3.0,0.0,2.0,Personal Auto,Personal L3,Offer1,Agent,511.200000,Four-Door Car,Medsize
41,MB98372,Oregon,2771.045031,Yes,Basic,College,2011-11-01 00:00:00,Employed,F,50071.0,...,32.0,0.0,1.0,Corporate Auto,Corporate L3,Offer2,Agent,18.918935,Two-Door Car,Medsize
42,IL19217,California,3939.006370,Yes,Extended,Bachelor,1/28/11,Employed,F,60021.0,...,28.0,0.0,1.0,Personal Auto,Personal L2,Offer2,Agent,882.871945,Four-Door Car,Small
51,PY51963,California,33473.349460,Yes,Basic,Bachelor,2011-11-01 00:00:00,Employed,F,33190.0,...,95.0,0.0,2.0,Corporate Auto,Corporate L1,Offer1,Web,508.800000,SUV,Medsize
60,CD86811,Arizona,4458.113369,Yes,Basic,Bachelor,2011-02-01 00:00:00,Retired,M,17622.0,...,36.0,1.0,3.0,Personal Auto,Personal L1,Offer1,Agent,312.000000,Four-Door Car,Medsize
68,GW33762,Oregon,6846.150323,Yes,Extended,Bachelor,1/27/11,Unemployed,F,0.0,...,20.0,0.0,5.0,Personal Auto,Personal L3,Offer1,Call Center,456.000000,Two-Door Car,Large


In [152]:
data["State"].value_counts()

Oregon        370
California    369
Arizona       240
Nevada        119
Washington    107
Name: State, dtype: int64